# Web_crawler 
This is a web-crawler created by Bill Chang, and it is made for getting the flight data from website. 

The purpose is to collect the data for d3.js visualization project, and it is only for personal practice and education. 

## Read the html Using BeautifulSoup

import urllib to read url content

Note that Python3 does not read the **html** code as a string but as a bytearray, so we need to convert it to one with **decode.**

In [1]:
import urllib
from bs4 import BeautifulSoup
import urllib.request

def create_soup(url):
    #url = input('Enter the url -') #enter the url
    byte_arrary = urllib.request.urlopen(url)
    content_bytes = byte_arrary.read()
    html = content_bytes.decode("utf8")
    byte_arrary.close()
    return BeautifulSoup(html)
    #use beautifulsoup to read and get the data we want


In [2]:
#print ('The title of page is: \n',soup.title.string)

## Find out the data

According to the data, we can find that there are two kinds of airplane_schedule.

One kind is direct, another is non-direct.
Hence, we have to carefully extract the data.

In [3]:
def week(schedule,day,index):
    if schedule[index].find('i') == None:
        return 'No'
    else:
        return 'Yes'

def element_get(data):
    # crawl the data by tag selector
    dep_time = data.select('span.schedules__departure-time')[0].contents[0]
    dep_place = data.select('span.schedules__departure-code')[0].contents[0]
    arr_time = data.select('span.schedules__arrival-time')[0].contents[0]
    arr_place = data.select('span.schedules__arrival-code')[0].contents[0]
    #fli_dur = data.select('div.schedules__item.schedules__duration')[0].contents[0].strip()
    fli_code = data.select('span.schedules__airline-code')[0].contents[2].strip()
    aircraft = data.select('span.schedules__aircraft-code')[0].contents[0]
  
    try:
        alt = 0
        wait = data.select('div.schedules__item.schedules__transit-airport')[0].contents[0].strip()
    except IndexError:
        alt = 1
        Mon = None
        Tus = None
        Wed = None
        Thur = None
        Fri = None
        Sat = None
        Sun = None
        week_schedule = data.find('div',{'class':"schedules__item schedules__week"}).contents

        Mon = week(week_schedule,Mon,1)
        Tus = week(week_schedule,Tus,3)
        Wed = week(week_schedule,Wed,5)
        Thur = week(week_schedule,Thur,7)
        Fri = week(week_schedule,Fri,9)
        Sat = week(week_schedule,Sat,11)
        Sun = week(week_schedule,Sun,13)
    if alt == 1:
        return [fli_code,dep_time,dep_place,arr_time,arr_place,aircraft,[Mon,Tus,Wed,Thur,Fri,Sat,Sun]]
    if alt == 0:
        return [fli_code,dep_time,dep_place,arr_time,arr_place,aircraft]

In [4]:
def title_get(data):
    # function to get the title on the web
    title_table = data.select('nav.breadcrumbs.breadcrumbs-schedules-city_to_city.breadcrumbs-schedules')
    title_list = title_table[0].select('span[itemprop="title"]')

    title_part1 = title_list[2].contents[0]
    title_part2 = title_list[3].contents[0]
    title_part3 = title_list[4].contents[0].split()[3]
    title_part4 = title_list[5].contents[0].strip()

    title = '('+title_part1+'-'+title_part2+') '+'到'+' ('+title_part3+'-'+title_part4+')'
    return title

def data_dict(soup):
    #flight table preprocessing
    #title = [title_get(soup)]
    table = soup.select('li.schedules__data-list.js-schedule-item')
    #table2 = soup.select('li.schedules__data-list.js-schedule-item.schedule__non-direct.js-schedule-non-direct.hide')
    #table3 = soup.select('li.schedules__data-list.js-schedule-item.js-schedule-non-direct.hide')
    #table4 = soup.select('li.schedules__data-list.js-schedule-item.schedule__non-direct.js-schedule-non-direct-codeshare.hide')

    list1 = []
    #list2 = []
    #list3 = []
    #list4 = []
    for i in table:
        list1.append(element_get(i))
    '''
    for i in table2:
        list2.append(element_get(i))

    for i in table3:
        list3.append(element_get(i))
    
    for i in table4:
        list4.append([title_get(soup)]+element_get(i))    
    ''' 
    #final_table = []
    final_dict = {}
    # i not in list2 and i not in list3 and 
    '''
    for i in list1:
        if i not in list4:
            final_table.append(i)
    '''
    for i in list1:
        if i[0] not in final_dict:
            final_dict[i[0]] = i
    return final_dict

## Output the data to csv

When we write the data, remember to add **newline=''** to solve it write one more line in the csv.

Use writerows to write many lines in only one method.

In open, if we want to append data to old file, use 'a' to replace 'w'.

In [17]:
import csv

def write_data(csv_name,input_dict):
    with open(csv_name,'w',newline='') as myfile:   
        wr = csv.writer(myfile,delimiter=',',quoting=csv.QUOTE_ALL)
        wr.writerows(input_dict.values())
        
def test_read_data():
    # check the data can be read correctly
    new_list=[]
    with open ('test.csv','r') as myfile:
        rd = csv.reader(myfile,delimiter=',',quoting=csv.QUOTE_ALL)
        for row in rd:
            new_list.append(row)
    return new_list

In [6]:
import pickle
def data_append_csv():
    # get website data (the link got from first crawler)
    with open('web_list.txt','rb') as f:  # read the url list (get from crawler1)
        link_list = pickle.load(f)
    
    step = 1
    total = len(link_list)
    
    for link in link_list:    
        step += 1
        percent = round((step/total),3)*100
        print ('-',percent,'%-',sep='',end='')
        my_soup = create_soup(link)
        my_dict = data_dict(my_soup)
        write_data(my_dict)        

In [18]:
#data_append_csv()
test_list = test_read_data()
test_dict = {}

for inner_list in test_list:
    test_dict[inner_list[0]] = inner_list
for inner_list in test_list:
    if len(inner_list) == 7:
        test_dict[inner_list[0]] = inner_list
write_data('test2.csv',test_dict)